In [1]:
from bs4 import BeautifulSoup
from urlparse import urlparse

import zlib
import base64

import codecs
import operator

from os import listdir

In [2]:
DATA_PATH = "./data/all/"

In [3]:
urls_indexes_source = dict()

In [4]:
n_docs_total = 0

with open(DATA_PATH + "urls.txt", "r") as f_name:
    for pair in f_name:
        pair = pair.strip().split('\t')
        if len(pair) == 0:
            continue
        
        urls_indexes_source[int(pair[0])] = pair[1]
        n_docs_total += 1

In [5]:
def parse_url(url):  
    try:
        url_parsed = urlparse(url)
    except ValueError:
        return None
        
    if url_parsed.scheme in ["mailto"]:
        return None
    if not url_parsed.netloc or url_parsed.netloc == "lenta.ru":
        return url_parsed.path
    else:
        return None

In [6]:
n_indexes = 0
urls_indexes = dict()
urls_adjacency_list = dict()

In [7]:
parts = sorted([part for part in listdir(DATA_PATH) if "docs" in part])
n_docs_processed = 0

In [8]:
for part in parts:
    with open(DATA_PATH + part, "r") as f_name:        
        for pair in f_name:
            pair = pair.strip().split('\t')
        
            if len(pair) == 0:
                continue
            
            pair[0] = int(pair[0])
            try:
                pair[1] = zlib.decompress(base64.b64decode(pair[1]))
            except zlib.error:
                continue
            except TypeError:
                continue
                
            url_from = parse_url(urls_indexes_source[pair[0]])
            if (url_from is None):
                continue
                
            soup = BeautifulSoup(pair[1], "html.parser")
            url_to = [parse_url(content['href']) for content in soup.find_all('a', href=True)]
            
            for url in [url_from] + url_to:
                if url and url not in urls_indexes:
                    urls_indexes[url] = n_indexes
                    n_indexes += 1
            
            index_to = sorted(set([urls_indexes[url] for url in url_to if url]))
            index_from = urls_indexes[url_from]
        
            if len(index_to):
                urls_adjacency_list[index_from] = index_to
            
            n_docs_processed += 1
            
            if n_docs_processed % 50 == 0:
                print u"\r{}: {} docs of {} are processed...".format(part, n_docs_processed, n_docs_total),

print u"\r{} docs are processed...".format(n_docs_processed)

564538 docs are processed...


In [9]:
print u"Чиcло вершин в графе: {}".format(n_indexes)
print u"Чиcло висячих вершин в графе: {}".format(n_indexes - len(urls_adjacency_list))

Чиcло вершин в графе: 1149606
Чиcло висячих вершин в графе: 589670


In [10]:
with codecs.open("./data/urls.index", mode="w", encoding="utf-8") as f_name:
    urls_indexes = sorted(urls_indexes.iteritems(), key=operator.itemgetter(1))
    for url, index in urls_indexes:
        f_name.write(u"{}\t{}\n".format(index, url))

In [11]:
with codecs.open("./data/urls.pairs", mode="w", encoding="utf-8") as f_name:
    urls_adjacency_list = sorted(urls_adjacency_list.iteritems(), key=operator.itemgetter(0)) 
    for index_from, index_to in urls_adjacency_list:
        f_name.write('{}\t'.format(index_from))
        for index in index_to:
            f_name.write('{} '.format(index))
        f_name.write('\n')